In [1]:
!apt-get update -y
!apt-get install -y poppler-utils

!pip install -q \
  numpy==1.26.4 \
  faiss-cpu \
  sentence-transformers \
  bitsandbytes accelerate \
  git+https://github.com/huggingface/transformers \
  pdf2image pillow \
  pymupdf pdfplumber

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cli.github.com/packages stable InRelease [3,917 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,225 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 https://cli.github.com/packages stable/main amd64 Packages [345 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packa

In [1]:
import os, re, gc, math
from dataclasses import dataclass
from typing import List, Tuple, Dict, Optional

import numpy as np
import faiss
import torch

import fitz  # pymupdf
import pdfplumber
from pdf2image import convert_from_path
from PIL import Image

from sentence_transformers import SentenceTransformer

from transformers import AutoProcessor, AutoModelForImageTextToText, BitsAndBytesConfig

In [2]:
import os
os.environ["TRANSFORMERS_VERBOSITY"] = "error"

In [3]:
MODEL_ID = "Qwen/Qwen3-VL-4B-Instruct"

bnb = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

print("🔧 Qwen3-VL yükleniyor...")
model = AutoModelForImageTextToText.from_pretrained(
    MODEL_ID,
    device_map="auto",
    quantization_config=bnb,
    trust_remote_code=True
)
processor = AutoProcessor.from_pretrained(MODEL_ID)

torch.cuda.empty_cache()
gc.collect()
print("✅ Model hazır")


🔧 Qwen3-VL yükleniyor...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

video_preprocessor_config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

✅ Model hazır


In [4]:
from google.colab import files
uploaded = files.upload()  # PDF(ler)i seç
pdf_paths = list(uploaded.keys())
print("Yüklenen PDF'ler:", pdf_paths)


Saving 1706.03762v7.pdf to 1706.03762v7.pdf
Yüklenen PDF'ler: ['1706.03762v7.pdf']


In [25]:
@dataclass
class Chunk:
    text: str
    pdf: str
    page: int
    kind: str  # "text" | "table" | "page"

def clean_text(s: str) -> str:
    s = re.sub(r"\s+", " ", s or "").strip()
    return s

def split_into_chunks(text: str, max_chars: int = 1200, overlap: int = 150) -> List[str]:
    text = clean_text(text)
    if not text:
        return []
    chunks = []
    i = 0
    while i < len(text):
        j = min(len(text), i + max_chars)
        chunk = text[i:j].strip()
        if chunk:
            chunks.append(chunk)
        if j == len(text):
            break
        i = max(0, j - overlap)
    return chunks

def extract_page_texts(pdf_path: str) -> List[str]:
    doc = fitz.open(pdf_path)
    pages = []
    for i in range(doc.page_count):
        t = doc.load_page(i).get_text("text")
        pages.append(clean_text(t))
    doc.close()
    return pages

def extract_tables(pdf_path: str) -> Dict[int, List[str]]:
    """
    pdfplumber ile tablolari çıkarır.
    Her sayfa için tablo(lar) -> string list.
    """
    out: Dict[int, List[str]] = {}
    with pdfplumber.open(pdf_path) as pdf:
        for pi, page in enumerate(pdf.pages, start=1):
            try:
                tables = page.extract_tables() or []
            except Exception:
                tables = []
            table_texts = []
            for t in tables:
                # t: list[list[str]]
                rows = []
                for row in t:
                    row = [clean_text(x) for x in (row or [])]
                    if any(row):
                        rows.append(" | ".join([x if x else "" for x in row]))
                if rows:
                    table_texts.append("\n".join(rows))
            if table_texts:
                out[pi] = table_texts
    return out

all_text_chunks: List[Chunk] = []
all_table_chunks: List[Chunk] = []
all_page_chunks: List[Chunk] = []  # image retriever için "caption" gibi

for pdf in pdf_paths:
    page_texts = extract_page_texts(pdf)     # sayfa bazlı text
    page_tables = extract_tables(pdf)        # sayfa bazlı tablolar

    for page_idx, page_text in enumerate(page_texts, start=1):
        # PAGE chunk (image retriever skoru için)
        if page_text:
            all_page_chunks.append(Chunk(
                text=f"[PAGE TEXT] {page_text[:3000]}",
                pdf=pdf,
                page=page_idx,
                kind="page"
            ))

        # TEXT chunks
        for ch in split_into_chunks(page_text, max_chars=1200, overlap=150):
            all_text_chunks.append(Chunk(
                text=ch,
                pdf=pdf,
                page=page_idx,
                kind="text"
            ))

        # TABLE chunks
        if page_idx in page_tables:
          # Tüm tabloları tek bir büyük chunk olarak birleştir
          combined_tables = []

          for ti, table_str in enumerate(page_tables[page_idx], start=1):
              table_str = table_str.strip()
              if table_str:
                  combined_tables.append(f"[TABLE {ti}]\n{table_str}")

          if combined_tables:
              big_table_chunk = "\n\n".join(combined_tables)

              all_table_chunks.append(Chunk(
                  text=big_table_chunk,
                  pdf=pdf,
                  page=page_idx,
                  kind="table"
              ))


print("✅ Chunk sayıları")
print("  text :", len(all_text_chunks))
print("  table:", len(all_table_chunks))
print("  page :", len(all_page_chunks))


✅ Chunk sayıları
  text : 41
  table: 5
  page : 15


In [26]:
embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

def embed_texts(texts: List[str], batch_size: int = 32) -> np.ndarray:
    vecs = embedder.encode(texts, batch_size=batch_size, convert_to_numpy=True, normalize_embeddings=True)
    return vecs.astype(np.float32)

def build_faiss_index(chunks: List[Chunk]) -> Tuple[faiss.IndexFlatIP, np.ndarray]:
    if not chunks:
        idx = faiss.IndexFlatIP(384)
        meta = np.empty((0,), dtype=object)
        return idx, meta
    texts = [c.text for c in chunks]
    vecs = embed_texts(texts)
    idx = faiss.IndexFlatIP(vecs.shape[1])  # cosine = IP (normalize edildi)
    idx.add(vecs)
    meta = np.array(chunks, dtype=object)
    return idx, meta

text_index, text_meta = build_faiss_index(all_text_chunks)
table_index, table_meta = build_faiss_index(all_table_chunks)
page_index, page_meta = build_faiss_index(all_page_chunks)

print("✅ FAISS index hazır")


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


✅ FAISS index hazır


In [7]:
_page_img_cache: Dict[Tuple[str,int], Image.Image] = {}

def get_page_image(pdf_path: str, page: int, dpi: int = 170) -> Image.Image:
    key = (pdf_path, page)
    if key in _page_img_cache:
        return _page_img_cache[key]
    imgs = convert_from_path(pdf_path, first_page=page, last_page=page, dpi=dpi)
    img = imgs[0].convert("RGB")
    _page_img_cache[key] = img
    return img


In [40]:
TABLE_HINTS = ["table", "tablo", "value", "values", "time", "cure", "hours", "minutes", "°c", "percent", "%", "spec", "property", "properties"]
FIG_HINTS   = ["figure", "fig.", "diagram", "image", "illustration", "shown", "plot", "graph"]
NUM_HINTS   = ["how many",  "value", "time",  "hour", "minute", "°c", "%", "rate"]

def hint_score(q: str, hints: List[str]) -> float:
    ql = q.lower()
    return sum(1.0 for h in hints if h in ql)

def search_index(index: faiss.IndexFlatIP, meta: np.ndarray, q: str, top_k: int = 3):
    qv = embed_texts([q])
    if index.ntotal == 0:
        return []
    scores, ids = index.search(qv, top_k)
    res = []
    for s, i in zip(scores[0], ids[0]):
        if i == -1:
            continue
        res.append((float(s), meta[i]))
    return res

def qwen_generate(system_text: str, user_text: str, images: Optional[List[Image.Image]] = None, max_new_tokens: int = 512) -> str:
    # messages -> text prompt
    messages = [
        {"role": "system", "content": [{"type": "text", "text": system_text}]},
        {"role": "user", "content": [{"type": "text", "text": user_text}]},
    ]
    prompt = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    if images:
        inputs = processor(text=[prompt], images=images, return_tensors="pt", padding=True).to(model.device)
    else:
        inputs = processor(text=[prompt], return_tensors="pt", padding=True).to(model.device)

    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False
        )

    # decode only generated part
    gen = out[:, inputs["input_ids"].shape[1]:]
    return processor.batch_decode(gen, skip_special_tokens=True)[0].strip()

def adaptive_answer(question: str, top_k: int = 3) -> str:
    # 1) paralel retrieval
    text_hits  = search_index(text_index,  text_meta,  question, top_k=top_k)
    table_hits = search_index(table_index, table_meta, question, top_k=top_k)
    page_hits  = search_index(page_index,  page_meta,  question, top_k=top_k)

    best_text  = text_hits[0][0]  if text_hits  else -1.0
    best_table = table_hits[0][0] if table_hits else -1.0
    best_page  = page_hits[0][0]  if page_hits  else -1.0

    # 2) ipucu ağırlığı
    table_bias = hint_score(question, TABLE_HINTS) + hint_score(question, NUM_HINTS)
    fig_bias   = hint_score(question, FIG_HINTS)

    # 3) karar: skor + bias
    # page (görsel) ancak gerçekten güçlü ise veya fig_bias varsa seçilsin
    q_lower = question.lower()

    table_bias = 0.0
    if any(k in q_lower for k in [
        "table", "according to table", "refer to table",
        "compare", "comparison", "values", "results in table",
        "table 1", "table 2"
    ]):
        table_bias = 0.20  # kritik değer

    score_map = {
        "text":  best_text,
        "table": best_table + table_bias,
        "image": best_page
    }

    channel = decide_channel(question, best_text, best_table, best_page)



    # 4) yakınsa hybrid (2 kanalı birlikte verelim)
    scores_sorted = sorted(score_map.items(), key=lambda x: x[1], reverse=True)
    top1, top2 = scores_sorted[0], scores_sorted[1]
    hybrid = (top1[1] - top2[1]) < 0.03  # çok yakınsa

    # 5) kanıt derle
    def format_evidence(hit_list, title: str, max_chars: int = 1800) -> str:
        if not hit_list:
            return f"{title}: (couldn't find)\n"
        lines = [f"{title}:"]
        for s, ch in hit_list[:top_k]:
            lines.append(f"- PDF: {os.path.basename(ch.pdf)} | Page: {ch.page} | score={s:.3f}\n  {ch.text[:max_chars]}")
        return "\n".join(lines) + "\n"

    sys = (
        "You are a retrieval-augmented generation assistant. "
        "Answer ONLY using the provided evidence. "
        "If the evidence is insufficient, say 'I don't know' and explain what is missing. "
        "Always write the answer in ENGLISH and cite the source page(s) at the end."
    )

    # 6) seçilen akış
    if channel == "image" and page_hits:
        # en iyi sayfayı görsele çevir
        best = page_hits[0][1]
        img = get_page_image(best.pdf, best.page)

        # hybrid ise text/table’dan da kanıt ekle
        evidence = ""
        if hybrid:
            evidence += format_evidence(text_hits,  "TEXT EVIDENCE")
            evidence += format_evidence(table_hits, "TABLE EVIDENCE")
        evidence += format_evidence(page_hits, "PAGE EVIDENCE")

        user = f"QUESTION: {question}\n\n{evidence}\nWrite a clear and concise answer in ENGLISH."
        ans = qwen_generate(sys, user, images=[img])

        # kaynak ekini güçlendirelim
        src = f"\n\SOURCE: {os.path.basename(best.pdf)} | Page {best.page}"
        return ans + src

    elif channel == "table" and table_hits:
        evidence = format_evidence(table_hits, "TABLE EVIDENCE")
        if hybrid:
            evidence += format_evidence(text_hits, "TEXT EVIDENCE")
            evidence += format_evidence(page_hits, "PAGE EVIDENCE")

        user = f"QUESTION: {question}\n\n{evidence}\nWrite a clear and concise answer in ENGLISH."
        best = table_hits[0][1]
        ans = qwen_generate(sys, user, images=None)
        src = f"\n\nSOURCE: {os.path.basename(best.pdf)} | Page {best.page}"
        return ans + src

    else:
        evidence = format_evidence(text_hits, "TEXT EVIDENCE")
        if hybrid:
            evidence += format_evidence(table_hits, "TABLE EVIDENCE")
            evidence += format_evidence(page_hits, "PAGE EVIDENCE")

        user = f"QUESTION: {question}\n\n{evidence}\nWrite a clear and concise answer in ENGLISH."
        best = text_hits[0][1] if text_hits else (page_hits[0][1] if page_hits else None)
        ans = qwen_generate(sys, user, images=None)
        if best:
            ans += f"\n\nSOURCE: {os.path.basename(best.pdf)} | Page {best.page}"
        return ans


<>:119: SyntaxWarning: invalid escape sequence '\S'
<>:119: SyntaxWarning: invalid escape sequence '\S'
/tmp/ipython-input-298903847.py:119: SyntaxWarning: invalid escape sequence '\S'
  src = f"\n\SOURCE: {os.path.basename(best.pdf)} | Page {best.page}"


In [41]:
def decide_channel(question, best_text, best_table, best_image):
    q_lower = question.lower()

    table_bias = 0.0
    if any(k in q_lower for k in [
        "table", "according to table", "refer to table",
        "compare", "comparison", "values",
        "table 1", "table 2"
    ]):
        table_bias = 0.20

    score_map = {
        "text":  best_text,
        "table": best_table + table_bias,
        "image": best_image
    }

    return max(score_map, key=score_map.get)


In [ ]:
while True:
    q = input("\nWrite a question ('q' for quit): ").strip()
    if q.lower() == "q":
        print("Quitting.")
        break
    print("\nAnswer:\n", adaptive_answer(q, top_k=3))


## Performans Değerlendirmesi

In [42]:
EVAL_QUESTIONS = [
    {
        "question": "According to the Introduction section of the paper, what are the main limitations of recurrent neural networks (RNNs) in sequence modeling, and how does the Transformer architecture address these limitations using self-attention?",
        "expected_channel": "text",
        "expected_page": 1
    },
    {
        "question": "Describe the encoder–decoder structure of the Transformer, including the role of self-attention, feed-forward layers, residual connections, and layer normalization. How do these components interact within a single layer?",
        "expected_channel": "image",
        "expected_page": 3
    },
    {
        "question": "Using Figure 2, explain how Multi-Head Attention differs from single-head attention. Why does using multiple heads allow the model to attend to different representation subspaces?",
        "expected_channel": "image",
        "expected_page": 4
    },
    {
        "question": "Referring to Table 1, compare self-attention and recurrent layers in terms of sequential operations and maximum path length. Why does a shorter path length make learning long-range dependencies easier?",
        "expected_channel": "table",
        "expected_page": 6
    },
    {
        "question": "Why does the sequential nature of recurrent neural networks limit parallelization during training, and how does this affect scalability for long sequences?",
        "expected_channel": "text",
        "expected_page": 2
    },
    {
        "question": "Explain the concept of self-attention as described in Section 2. How does it enable modeling dependencies regardless of distance in the input sequence?",
        "expected_channel": "text",
        "expected_page": 2
    },
    {
        "question": "What problem does positional encoding solve in the Transformer architecture, and why are sinusoidal functions used instead of learned embeddings?",
        "expected_channel": "text",
        "expected_page": 6
    },
    {
        "question": "Based on Figure 1, explain how information flows through the encoder and decoder stacks in the Transformer architecture.",
        "expected_channel": "image",
        "expected_page": 3
    },
    {
        "question": "Using the attention visualization figures, what evidence suggests that different attention heads specialize in different linguistic or structural roles?",
        "expected_channel": "image",
        "expected_page": 13
    },
    {
        "question": "According to Table 1, how does the computational complexity of self-attention compare to convolutional layers as sequence length increases?",
        "expected_channel": "table",
        "expected_page": 6
    },
    {
        "question": "Using Table 2, compare the BLEU scores and training costs of the Transformer model against previous state-of-the-art models. What trade-offs are observed?",
        "expected_channel": "table",
        "expected_page": 8
    },
    {
        "question": "Referring to Table 3, how does increasing the number of attention heads affect model performance, and what conclusion do the authors draw from this ablation study?",
        "expected_channel": "table",
        "expected_page": 9
    }
]


In [43]:
import pandas as pd

def evaluate_system(eval_questions, top_k=3):
    results = []

    for item in eval_questions:
        q = item["question"]
        exp_channel = item["expected_channel"]
        exp_page = item["expected_page"]

        # --- PARALLEL RETRIEVAL (NO LLM GENERATION) ---
        text_hits  = search_index(text_index,  text_meta,  q, top_k)
        table_hits = search_index(table_index, table_meta, q, top_k)
        page_hits  = search_index(page_index,  page_meta,  q, top_k)


        best_text  = text_hits[0][0]  if text_hits  else -1
        best_table = table_hits[0][0] if table_hits else -1
        best_image = page_hits[0][0]  if page_hits  else -1


        predicted_channel = decide_channel(q, best_text, best_table, best_image)

        # --- Hit@k ---
        retrieved_pages = []
        for hits in [text_hits, table_hits, page_hits]:
            retrieved_pages += [h[1].page for h in hits]

        hit_at_1 = int(exp_page in retrieved_pages[:1])
        hit_at_3 = int(exp_page in retrieved_pages[:3])

        # --- Channel accuracy ---
        channel_correct = int(predicted_channel == exp_channel)

        results.append({
            "question": q,
            "expected_channel": exp_channel,
            "predicted_channel": predicted_channel,
            "channel_correct": channel_correct,
            "hit@1": hit_at_1,
            "hit@3": hit_at_3
        })

    return pd.DataFrame(results)


In [44]:
df_results = evaluate_system(EVAL_QUESTIONS, top_k=1)
df_results


,question,expected_channel,predicted_channel,channel_correct,hit@1,hit@3
0,According to the Introduction section of the p...,text,text,1,0,0
1,Describe the encoder–decoder structure of the ...,image,image,1,1,1
2,"Using Figure 2, explain how Multi-Head Attenti...",image,image,1,0,0
3,"Referring to Table 1, compare self-attention a...",table,text,0,1,1
4,Why does the sequential nature of recurrent ne...,text,text,1,1,1
5,Explain the concept of self-attention as descr...,text,text,1,0,0
6,What problem does positional encoding solve in...,text,text,1,1,1
7,"Based on Figure 1, explain how information flo...",image,image,1,1,1
8,"Using the attention visualization figures, wha...",image,image,1,1,1
9,"According to Table 1, how does the computation...",table,text,0,1,1


In [45]:
import pandas as pd
from sklearn.metrics import f1_score, classification_report

# ---------- Channel-level metrics ----------
y_true = df_results["expected_channel"]
y_pred = df_results["predicted_channel"]

channel_accuracy = df_results["channel_correct"].mean()
macro_f1 = f1_score(y_true, y_pred, average="macro")
weighted_f1 = f1_score(y_true, y_pred, average="weighted")

# ---------- Retrieval metrics ----------
hit_at_1 = df_results["hit@1"].mean()
hit_at_3 = df_results["hit@3"].mean()

def compute_mrr(df):
    rr = []
    for _, row in df.iterrows():
        if row["hit@1"] == 1:
            rr.append(1.0)
        elif row["hit@3"] == 1:
            rr.append(1/3)
        else:
            rr.append(0.0)
    return sum(rr) / len(rr)

mrr = compute_mrr(df_results)

# ---------- Summary table ----------
metrics_summary = pd.DataFrame({
    "Metric": [
        "Channel Accuracy",
        "Channel Macro F1",
        "Channel Weighted F1",
        "Hit@1",
        "Hit@3",
        "MRR"
    ],
    "Value": [
        channel_accuracy,
        macro_f1,
        weighted_f1,
        hit_at_1,
        hit_at_3,
        mrr
    ]
})

print("=== METRICS SUMMARY ===")
display(metrics_summary)

print("\n=== CHANNEL CLASSIFICATION REPORT ===")
print(classification_report(y_true, y_pred))


=== METRICS SUMMARY ===


,Metric,Value
0,Channel Accuracy,0.666667
1,Channel Macro F1,0.538721
2,Channel Weighted F1,0.538721
3,Hit@1,0.666667
4,Hit@3,0.750000
5,MRR,0.694444



=== CHANNEL CLASSIFICATION REPORT ===
              precision    recall  f1-score   support

       image       0.80      1.00      0.89         4
       table       0.00      0.00      0.00         4
        text       0.57      1.00      0.73         4

    accuracy                           0.67        12
   macro avg       0.46      0.67      0.54        12
weighted avg       0.46      0.67      0.54        12



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [13]:
import torch, gc
torch.cuda.empty_cache()
gc.collect()


166470

### **Dikkat!**
Aşağıdaki cell'de görüyoruz ki table skorları aslında 0 değil ancak performans ölçümlerinde bu şekilde görünüyor. Bunun nedeni kullanılan akadmeik çalışmada zaten çoğu table text olarak daha kapsamlı açıklandığı için text skorları daha yüksek değerler alıyor. Bu sebeple hiçbir table bilgisi table olarak işaretlenmiyor.

In [46]:
for item in EVAL_QUESTIONS:
    q = item["question"]

    text_hits  = search_index(text_index,  text_meta,  q, top_k=3)
    table_hits = search_index(table_index, table_meta, q, top_k=3)
    page_hits  = search_index(page_index,  page_meta,  q, top_k=3)

    best_text  = text_hits[0][0]  if text_hits  else -1
    best_table = table_hits[0][0] if table_hits else -1
    best_image = page_hits[0][0]  if page_hits  else -1

    print("\nQUESTION:", q)
    print(f"Scores → text: {best_text:.3f}, table: {best_table:.3f}, image: {best_image:.3f}")



QUESTION: According to the Introduction section of the paper, what are the main limitations of recurrent neural networks (RNNs) in sequence modeling, and how does the Transformer architecture address these limitations using self-attention?
Scores → text: 0.738, table: 0.141, image: 0.621

QUESTION: Describe the encoder–decoder structure of the Transformer, including the role of self-attention, feed-forward layers, residual connections, and layer normalization. How do these components interact within a single layer?
Scores → text: 0.792, table: 0.211, image: 0.805

QUESTION: Using Figure 2, explain how Multi-Head Attention differs from single-head attention. Why does using multiple heads allow the model to attend to different representation subspaces?
Scores → text: 0.691, table: 0.252, image: 0.694

QUESTION: Referring to Table 1, compare self-attention and recurrent layers in terms of sequential operations and maximum path length. Why does a shorter path length make learning long-ran